In [1]:
!pip install -q streamlit
!pip install streamlit-authenticator
!npm install localtunnel
!pip install googlesearch-python
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.4/699.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 14.3 MB/s eta 0:00:00
  Created wheel for streamlit-authenticator: filename=streamlit_authenticator-0.2.3-py3-none-any.whl size=10967 sha256=7a0fda1ae6da24eb322d04ab4e7a7c39353d39cd01ca3994cf99c3f9e26a1e62
  Stored in directory: /root/.cache/pip/wheels/fa/eb/54/009e28abdb72f6bab333aff4549375b8c89185ea1d6e034109
Successfully built streamlit-authenticator
npm WARN saveError ENOENT: no such file or directory, open '/co

In [2]:
%%writefile app.py

import streamlit as st
from googlesearch import search
from bs4 import BeautifulSoup
import requests
# Function to perform Google search and retrieve top links
def perform_google_search(query):
    links = []
    for link in search(query, num_results=20):
        links.append(link)
        if len(links) == 20:
            break
    return links
# Function to scrape article content using BeautifulSoup
def scrape_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    content = soup.get_text()
    return content

# Function to count keyword occurrences in article content
def count_keyword_occurrences(content, keyword):
    return content.lower().count(keyword.lower())

def categorize_articles(average_count, keyword_count):
    if 0 <= keyword_count <= average_count / 2:
        colour_code = 'red'
    elif average_count / 2 < keyword_count <= average_count:
        colour_code = 'orange'
    else:
        colour_code = 'green'
    return colour_code

# Function to display article information
def display_article_info(keyword_count, title, meta_description, colour_code, link, keyword):
    st.write(
        f"""
        <div style="background-color: {colour_code}; padding: 10px; border-radius: 5px; margin-bottom: 10px;">
            <div style="display: flex; align-items: center;">
                <div>
                    <p>Keyword: {keyword}</p>
                    <p>Keyword Count: {keyword_count}</p>
                    <p>Title: <a href={link}>{title}</a></p>
                    <p>Meta Description: {meta_description}</p>
                </div>
            </div>
        </div>
        """,
        unsafe_allow_html=True
    )

def display_clue_cards():

    html_code = """
    <div style="display: grid; grid-template-columns: auto 1fr; align-items: center; grid-gap: 10px;">

      <div style="width: 20px; height: 20px; float: left; background-color: red;"> </div>
      <div style="margin-left: 6%;">
        Least number of keywords
      </div>

      <div style="width: 20px; height: 20px; background-color: orange;"> </div>
      <div style="margin-left: 6%;">
        Medium number of keywords
      </div>

      <div style="width: 20px; height: 20px; background-color: green;"> </div>
      <div style="margin-left: 6%;">
        Highest number of keywords
      </div>

    </div>
    """


    # Render HTML in Streamlit app
    st.markdown(html_code, unsafe_allow_html=True)


# Main function to run the Streamlit application
def main():
    st.title("Article Search")
    blog_content = st.text_area("Enter Blog Content")
    keywords = [each.strip() for each in st.text_input("Enter Keyword").split(',')]
    if st.button("Search"):
        links = perform_google_search(blog_content)
        cnt = 0
        output = {}
        counts = {}
        # Iterate through the links and scrape the article content
        for link in links:
            try:
              content = scrape_article_content(link)
              soup = BeautifulSoup(requests.get(link).content, 'html.parser')
              title = soup.title.string if soup.title else ""
              meta_description_tag = soup.find('meta', attrs={'name': 'description'})
              meta_description = meta_description_tag['content'] if meta_description_tag and 'content' in meta_description_tag.attrs else ""
              for keyword in keywords:
                  if ('been blocked' not in content) and ('unable to access' not in content) and ('Please enable cookies' not in content) and ('403 Forbidden' not in content)  and ('502 Server Error' not in content) and ('Just a moment' not in title) and ('Access Denied' not in title) and (title != ''):
                      keyword_count = count_keyword_occurrences(content, keyword)
                      if keyword in counts:
                          counts[keyword].append(count_keyword_occurrences(content, keyword))
                      else:
                          counts[keyword] = [count_keyword_occurrences(content, keyword)]

                      if link not in output:
                          output[link] = [{'title': title, 'meta_description': meta_description, 'content': content, 'keyword':keyword, 'keyword_count':keyword_count}]
                      else:
                          output[link].append({'title': title, 'meta_description': meta_description, 'content': content, 'keyword':keyword, 'keyword_count':keyword_count})
              if cnt >= 10:
                  break
              cnt += 1
            except Exception as e:
                pass
        # Calculate the average keyword count for each keyword
        average_count = {}
        for keyword in keywords:
            average_count[keyword] = sum(counts[keyword]) / len(counts[keyword])
        display_clue_cards()
        st.write("These are the top 10 articles along with the frequency of the keyword for the given blog content")
        # Iterate through the links and print the results
        for link in links:
            if link in output:
                for i in range(len(keywords)):
                    colour_code = categorize_articles(average_count[keywords[i]], output[link][i]['keyword_count'])
                    title = output[link][i]['title']
                    meta_description = output[link][i]['meta_description']
                    keyword = output[link][i]['keyword']
                    keyword_count = output[link][i]['keyword_count']


                    display_article_info(keyword_count, title, meta_description, colour_code, link, keyword)


if __name__ == "__main__":
    main()

Writing app.py


In [3]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.148.71.197


In [4]:
!streamlit run app.py --logger.level=debug &>/content/logs.txt &

In [5]:
!npx localtunnel --port 8501

npx: installed 22 in 3.389s
your url is: https://bumpy-tools-worry.loca.lt
^C
